In [ ]:
import json
import os

outdir = './output'
if not os.path.exists(outdir):
    os.mkdir(outdir)
with open('bus_routes.json', 'r') as f:
  data = json.load(f)

In [ ]:
service = "851"
direction = "1"
route = data[service][direction]

In [ ]:
od = []
for seq, id in enumerate(route[:-1]):
    next_stop = seq + 1
    for stop in route[next_stop:]:
        if id['code'] == stop['code']:
            break #we've come back to the same bus stop. assume all pax from here on out boarded on 2nd visit.
        od.append((int(id['code']), int(stop['code'])))

To use:
Copy origin_destination_summary from the data folders into the working directory, rename to "od.csv" or change filename below

This code also works with byhour files but requires manual fixing:
- Remove line 1 (it just says `TOTAL_TRIPS` which is useless to us)
- Replace this part of line 2 `TIME_PER_HOUR,,,0,1` with `ORIGIN_PT_CODE,DESTINATION_PT_CODE,DAY_TYPE,0,1`
- Delete line 3

In [ ]:
import pandas as pd
df_source = pd.read_csv("1d-od.csv")
#df_source = pd.read_csv("od.csv")
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.DataFrame()
for src, dest in od:
    df = pd.concat([df, df_source[(df_source['ORIGIN_PT_CODE'] == src) & (df_source['DESTINATION_PT_CODE'] == dest)]])

#retrieve the filtered report for the given service and direction
df.to_csv(f"output/{service}_{direction}.csv")

In [ ]:
df_wd = df[df["DAY_TYPE"] == "WEEKDAY"]
pd.pivot_table(df, index="ORIGIN_PT_CODE", columns="DESTINATION_PT_CODE")

In [ ]:
import math

# we keep only the first digit of the stop code as it generally conforms to geographical areas.
# second digit may also be useful in some cases but could create too much statistical noise.
# for example, both CCK and BP have a mix of stop codes in both 44000 and 45000 ranges
def rounder(x):
    return int(math.floor(x/1000)) * 1000

df_round = df
df_round.ORIGIN_PT_CODE = df_round.ORIGIN_PT_CODE.apply(rounder)
df_round.DESTINATION_PT_CODE = df_round.DESTINATION_PT_CODE.apply(rounder)
df_round = df_round.groupby(["DAY_TYPE", "ORIGIN_PT_CODE", "DESTINATION_PT_CODE"]).sum()
df_round.to_csv(f"output/{service}_{direction}_grouped.csv")

In [ ]:
files = ["190_1_time.csv", "190_2_time.csv", "972_1_time.csv", "960_1_time.csv","960_2_time.csv"]
df_cons = pd.DataFrame()
for fn in files:
    df_cons = pd.concat([df_cons, pd.read_csv(f"output/{fn}")])
df_cons = df_cons.drop_duplicates()
df_cons.to_csv("btl.csv")

In [ ]:
def rounder(x):
    return int(math.floor(x/1000)) * 1000

df_round = df_cons
df_round.ORIGIN_PT_CODE = df_round.ORIGIN_PT_CODE.apply(rounder)
df_round.DESTINATION_PT_CODE = df_round.DESTINATION_PT_CODE.apply(rounder)
df_round = df_round.groupby(["DAY_TYPE", "ORIGIN_PT_CODE", "DESTINATION_PT_CODE"]).sum()
df_round.to_csv(f"btl_grouped.csv")